# Assignment 2 - Building a decision tree

This is a skeleton of a decision tree classifier for the example data set in `data/example.csv`.

In [219]:
import csv
import math
from statistics import median, mode, mean
from collections import Counter
from enum import Enum
import numpy as np

#A = [[1,2,3],[1,2,3],[1,2,3]]

#test = set([A[idx][1] for idx in range(len(A))])
#print(test)
#print(A[:,0])
#test2 = {}
#test2[1] = []
#temp = np.where( A[:,0] <= 3 )
#print(temp[0])
#def test(x):
#    print(x)
#    for i in range(0,2):
#        test(i)
        
#test('test')
    
#test2[1].append(temp[0])
#listi = np.array(test2[1][0].tolist())
#print(listi)
#print(listi[0])
#l = [idx for idx in listi]
#print(l)

#ls = [1,1,1,2,2,2,3,4,5,5,8,8]
#print([ls.count(ls[i]) for i in range(len(ls))])

### Some simple type definitions.

In [220]:
class AttrType(Enum):
    cat = 0  # categorical (qualitative) attribute
    num = 1  # numerical (quantitative) attribute
    target = 2  # target label


class NodeType(Enum):
    root = 0
    internal = 1
    leaf = 2


class SplitType(Enum):
    bin = 0  # binary split
    multi = 1  # multi-way split

Also, some basic classes to represent an attribute, a spltting procedure, and a node.

In [221]:
class Attribute(object):
    def __init__(self, label, type):
        assert type in AttrType
        self.label = label
        self.type = type
        self.stat = None  # holds mean for numerical and mode for categorical attributes


class Splitting(object):
    def __init__(self, attr, infogain, split_type, cond, splits):
        self.attr = attr  # attribute ID (index in ATTR)
        self.infogain = infogain  # information gain if splitting is done on this attribute
        self.split_type = split_type  # one of SplitType
        self.cond = cond  # splitting condition, i.e., values on outgoing edges
        self.splits = splits  # list of training records (IDs) for each slitting condition


class Node(object):
    def __init__(self, id, type, parent_id, children=None, edge_value=None, val=None, split_type=None, split_cond=None,
                 infogain=None):
        self.id = id  # ID (same as the index in DT.model list)
        self.type = type  # one of NodeType
        self.parent_id = parent_id  # ID of parent node (None if root)
        self.children = children  # list of IDs of child nodes
        self.edge_value = edge_value  # the value of the incoming edge (only if not root node)
        self.val = val  # if root or internal node: the attribute that is compared at that node; if leaf node: 
        #the target value
        self.split_type = split_type  # one of SplitType
        self.split_cond = split_cond  # splitting condition (median value for binary splits on numerical values; otherwise a list of categorical values (corresponding to child nodes))
        self.infogain = infogain

    def append_child(self, node_id):
        self.children.append(node_id)

The input filename is hard-coded.

In [222]:
INFILE = "data/housing_price_train.csv"

The attribute labels types are hard-coded too (the same order as in the file!).

In [223]:
#Read from training data:
with open(INFILE) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    row1 = next(csv_reader)
    labels = []
    types = []
    ATTR = []
    i = 0
    for x in row1:
        if i>0: #skip first element (ID)
            labels.append(x)
        i=i+1
    
    types = [AttrType.num, AttrType.cat, AttrType.num, AttrType.num, AttrType.cat, AttrType.cat, AttrType.cat,
            AttrType.cat, AttrType.cat, AttrType.cat, AttrType.cat, AttrType.cat, AttrType.cat, AttrType.cat, AttrType.cat,
            AttrType.cat, AttrType.num, AttrType.num, AttrType.num, AttrType.num, AttrType.cat, AttrType.cat, AttrType.cat,
            AttrType.cat, AttrType.cat, AttrType.num, AttrType.cat, AttrType.cat, AttrType.cat, AttrType.cat, AttrType.cat,
            AttrType.cat, AttrType.cat, AttrType.num, AttrType.cat, AttrType.num, AttrType.num, AttrType.num, AttrType.cat,
            AttrType.cat, AttrType.cat, AttrType.cat, AttrType.num, AttrType.num, AttrType.num, AttrType.num, AttrType.num,
            AttrType.num, AttrType.num, AttrType.num, AttrType.num, AttrType.num, AttrType.cat, AttrType.num, AttrType.cat,
            AttrType.num, AttrType.cat, AttrType.cat, AttrType.num, AttrType.cat, AttrType.num, AttrType.num, AttrType.cat,
            AttrType.cat, AttrType.cat, AttrType.num, AttrType.num, AttrType.num, AttrType.num, AttrType.num, AttrType.num,
            AttrType.cat, AttrType.cat, AttrType.cat, AttrType.num, AttrType.num, AttrType.num, AttrType.cat, AttrType.cat,
            AttrType.target]
    
    for i in range(len(labels)):
        ATTR.append(Attribute(labels[i],types[i]))
        
    print(len(labels))
    print(len(ATTR))

80
80


The index of the target attribute (assuming it's the last).

In [224]:
IDX_TARGET = len(ATTR) - 1 
remAttrs = set(range(len(ATTR) - 1))

A main class DT representing the decision tree classifier. It could represent with methods:

  - a given impurity measure;
  - the search for the best attribute to split with;
  - the addition of a node to the tree;
  - a convenient model printer;
  - the recursive call for obtaining a tree;
  - a builder and an applier.

In [225]:
class DT(object):
    def __init__(self):
        self.data = None  # training data set (loaded into memory)
        self.model = None  # decision tree model
        self.default_target = 0.0  # default target class

    def __load_data(self):
        with open(INFILE) as csvfile:
            self.data = []
            csvreader = csv.reader(csvfile, delimiter=',')
            next(csvreader) #REMOVE FIRST ROW
            for row in csvreader:
                iterRow = iter(row)
                next(iterRow) #remove first column (IDS)
                row = list(iterRow)
                rec = []
                for i in range(len(ATTR)):
                    val = row[i].strip()
                    #print(val)
                    # convert numerical attributes
                    if val == 'NA':
                        val = -1
                    if ATTR[i].type == AttrType.num:  # Note that this will break for "?" (missing attribute)
                            val = float(val)
                    rec.append(val)
                self.data.append(rec)
                #self.data.pop(0) #REMOVE FIRST COLUMN (IDS)
                #for row in self.data:
                #    del row[0] #REMOVE FIRST ROW (LABELS)
                #print(self.data[0])
                # self.data.append([element.strip() for element in row])  # strip spaces


    
    def __mean_squared_error(self, records):
        """
        Calculates mean squared error for a selection of records.

        :param records: Data records (given by indices)
        """
        # TODO
        predictions = self.predict(records)
        sqe = np.zeros(len(records)) #square error
        i = 0
        for rec in records:
            sqe[i] = (self.data[records,IDX_TARGET]-predictions[records, IDX_TARGET])**2 
            i = i+1
        MSE = mean(sqe) #mean square error
        return MSE
    
    def __entropy(self, records, attribute):
        """
        Calculates entropy for a selection of records.

        :param records: Data records (given by indices)
        """
        data = []
        uniques = []
        ent = 0
        for rec in records:
            data.append(self.data[rec])
        col_vals=[row[IDX_TARGET] for row in data]
        counts = [col_vals.count(col_vals[i]) for i in range(len(col_vals))]
        for x in counts:
            if x not in uniques:
                uniques.append(x)
        ent = 0
        for x in uniques:
            temp = x/len(records)*math.log(x/len(records))
            ent = ent - temp
        return ent

    def __find_best_attr(self, attrs, records):
        """
        Finds the attribute with the largest gain.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :return:
        """
        #mse_p = self.__mean_squared_error(records)  # parent's MSE
        
        splittings = []  # holds the splitting information for each attribute
        numKey = 0
        for a in attrs:
            catKey = []
            assert ATTR[a].type in AttrType
            splits = {}  # record IDs corresponding to each split
            # splitting condition depends on the attribute type
            if ATTR[a].type == AttrType.target:  # skip target attribute
                continue
            elif ATTR[a].type == AttrType.cat:  # categorical attribute
                # multi-way split on each possible value
                split_mode = SplitType.multi
                # each possible attr value corresponds to a split (indexed with categorical labels)
                # Note: it's important to consider attr values from the entire training set
                split_cond = set([self.data[idx][a] for idx in range(len(self.data))])
                #print(len(split_cond))
                #print(split_cond)
                # TODO collect training records for each split 
                # `splits[val]` holds a list of records for a given split,
                # where `val` is an element of `split_cond`
                for val in split_cond:
                    splits[val] = [] #CATEGORICAL SPLIT
                #index 0 for np.where gir value og datatype:
                    data = [row[a] for row in self.data]
                    temp_i = np.where(np.asarray(data)==val)[0] #index 0 because index 1 in np.where returns datatype.
                    list_i = list(temp_i)
                    splits[val] = list_i
                    #print(val)
                    catKey.append(val)
                    
            elif ATTR[a].type == AttrType.num:  # numerical attribute => binary split on median value
                #print(ATTR[a].type)
                #print([row[a] for row in self.data])
                split_mode = SplitType.bin
                temp_med = [float(row[a]) for row in self.data]
                split_cond = median(temp_med)  # (i.e., if less or equal than this value)
                # TODO collect training records for each split (in `splits`)
                splits[split_cond] = [] #NUMERICAL SPLIT
                #index 0 for np.where gir value og datatype:
                data = [float(row[a]) for row in self.data]
                temp_l = np.where(np.asarray(data)<=split_cond)[0]
                temp_u = np.where(np.asarray(data)<=split_cond)[0]
                list_l = list(temp_l)
                list_u = list(temp_u)
                splits[split_cond].append(list_l) # index 0: <= median
                splits[split_cond].append(list_u) # index 1: > 
                numKey = split_cond
            # TODO compute gain for attribute a
            ent_p = self.__entropy(records,a)    #parent's entropy
            infogain = 0
            if len(splits)==1: #NUMERIC INFOGAIN
                p1 = len(splits[numKey][0])/len(records)
                p2 = len(splits[numKey][1])/len(records)
                #print(splits[numKey][0])
                ent_c1 = self.__entropy(splits[numKey][0],a)
                ent_c2 = self.__entropy(splits[numKey][1],a)
                infogain = ent_p - (p1*ent_c1+p2*ent_c2)
            elif len(splits)>1: #CATEGORICAL INFOGAIN
                for key in catKey:
                    #print(splits[key])
                    pi = len(splits[key])/len(records)
                    ent_ci = self.__entropy(splits[key],a)
                    infogain = infogain + pi*ent_ci
                infogain = ent_p - infogain
            #print(splits)
            splitting = Splitting(a, infogain, split_mode, split_cond, splits)
            splittings.append(splitting)

        # find best splitting (highest infogain is first element in reversed list)
        best_splitting = sorted(splittings, key=lambda x: x.infogain, reverse=True)[0]
        return best_splitting

    def __add_node(self, parent_id, node_type=NodeType.internal, edge_value=None, val=None, split_type=None,
                   split_cond=None):
        """
        Adds a node to the decision tree.

        :param parent_id:
        :param node_type:
        :param edge_value:
        :param val:
        :param split_type:
        :param split_cond:
        :return:
        """
        node_id = len(self.model)  # id of the newly assigned node
        if not self.model:  # the tree is empty
            node_type = NodeType.root

        node = Node(node_id, node_type, parent_id, children=[], edge_value=edge_value, val=val, split_type=split_type,
                    split_cond=split_cond)
        self.model.append(node)

        # also add it as a child of the parent node
        if parent_id is not None:
            self.model[parent_id].append_child(node_id)

        return node_id

    def __id3(self, attrs, records, parent_id=None, value=None):
        """
        Function ID3 that returns a decision tree.

        :param attrs: Set of attributes
        :param records: Training set (list of record ids)
        :param parent_id: ID of parent node
        :param value: Value corresponding to the parent attribute, i.e., label of the edge on which we arrived to this node
        :return:
        """
        # empty training set or empty set of attributes => create leaf node with default class
        if not records or not attrs:
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=self.default_target)
            return

        # if all records have the same target value => create leaf node with that target value
        same = all(self.data[idx][IDX_TARGET] == self.data[records[0]][IDX_TARGET] for idx in records)
        if same:
            target = self.data[records[0]][IDX_TARGET]
            self.__add_node(parent_id, node_type=NodeType.leaf, edge_value=value, val=target)
            return

        # find the attribute with the largest gain
        splitting = self.__find_best_attr(attrs, records)
        # add node
        node_id = self.__add_node(parent_id, edge_value=value, val=splitting.attr, split_type=splitting.split_type,
                                  split_cond=splitting.cond)
        
        # TODO call tree construction recursively for each split
        if splitting.attr in remAttrs:
            remAttrs.remove(splitting.attr)
        if len(remAttrs)>0:
            #print(splitting.splits)
            
            for key in splitting.splits: #FOR KEY IN SPLITTING.SPLITS??
                
                if key != -1 and self.model[node_id] != NodeType.leaf: #noen categorical verdier er -1 ('NA').
                    #CATEGORICAL:
                    if len(splitting.splits)>1:
                        #print(splitting.splits['Fa'])
                        #print(splitting.splits)
                        #print(key)
                        #print(splitting.splits)
                        self.__id3(remAttrs, [rec for rec in splitting.splits[key]], node_id, key)
                    #NUMERICAL:
                    elif len(splitting.splits)==1:
                        for i in range(2):
                            val = splitting.cond
                            #print(val)
                            self.__id3(remAttrs, [rec for rec in splitting.splits[key][i]], node_id, val)
        return
        
    def print_model(self, node_id=0, level=0):
        node = self.model[node_id]
        indent = "  " * level
        if node.type == NodeType.leaf:
            print(indent + str(node.edge_value) + " [Leaf node] class=" + str(node.val))
        else:
            cond = " <= " + str(node.split_cond) if ATTR[node.val].type == AttrType.num else " == ? "
            if node.type == NodeType.root:
                print("[Root node] '" + ATTR[node.val].label + "'" + cond)
            else:
                print(indent + str(node.edge_value) + " [Internal node] '" + ATTR[node.val].label + "'" + cond)
            # print tree for child notes recursively
            for n_id in node.children:
                self.print_model(n_id, level + 1)

    def build_model(self):
        self.__load_data()
        self.model = []  # holds the decision tree model, represented as a list of nodes
        # Get majority class
        #   Note: Counter returns a dictionary, most_common(x) returns a list with the x most common elements as
        #         (key, count) tuples; we need to take the first element of the list and the first element of the tuple 
        
        #l = [float(row) for row in range(self.data) if row[0] == 5]
        #print(l)
        #probs = []
        #data = self.data[:2]
        #print(self.data[:2]
        #col_vals=[float(col[IDX_TARGET]) for col in data]
        #for x in col_vals:
        #    if x not in probs:
        #        probs.append(x)
        #print(probs)
        
        
        target_vals=[float(col[IDX_TARGET]) for col in self.data]
        self.default_target = mean(target_vals)
        self.__id3(set(range(len(ATTR)-1)), list(range(len(self.data))))

    def apply_model(self, record):
        node = self.model[0]
        #while node.type != NodeType.leaf:
            
        	# TODO based on the value of the record's attribute that is tested in `node`,
        	# set `node` to one of its child nodes until a leaf node is reached
            
            #print("test")
        return node.val
    
    def predict(self, records):
        predictions = []
        for record in records:
            pred_val = self.apply_model(record)
            
            # TODO append pred_val to predictions
            
        return predictions
    
    

In [226]:
def createSubmission(test_ids, predictions):
    sub = pd.DataFrame()
    sub['Id'] = test_ids
    sub['SalePrice'] = predictions
    sub.to_csv('submission.csv',index=False)

Finally, the main function building a decision tree model, printing it and applying it on some unseen records.

In [227]:
def main():
    
    dt = DT()
    #l = [idx for idx in range(len(dt.__load_data())) if idx[0] == 1]
    #print(l)
    
    print("Build model:")
    dt.build_model()
    dt.print_model()

    print("\nApply model:")
    #print(dt.apply_model([1461,20,'RH',80,11622,'Pave','NA','Reg','Lvl','AllPub','Inside','Gtl','NAmes','Feedr','Norm','1Fam','1Story',5,6,1961,1961,
                          #'Gable','CompShg','VinylSd','VinylSd',None,0,'TA','TA','CBlock','TA','TA','No','Rec',468,'LwQ',144,270,882,'GasA','TA','Y',
                          #'SBrkr',896,0,0,896,0,0,1,0,2,1,'TA',5,'Typ',0,'NA','Attchd',1961,'Unf',1,730,'TA','TA','Y',140,0,0,0,120,0,'NA',
                          #'MnPrv','NA',0,6,2010,'WD','Normal']))
    
    #print(dt.apply_model(['sunny', 85, 85, 'false']))
    #print(dt.apply_model(['overcast', 75, 85, 'true']))
    #print(dt.apply_model(['rain', 75, 85, 'false']))

    #createSubmission(test_ids, dt.predict(test_data))

if __name__ == "__main__":
    main()

Build model:


C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:107: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


[Root node] 'PoolQC' == ? 
  Fa [Internal node] 'MiscFeature' == ? 
    Othr [Internal node] 'Alley' == ? 
      Pave [Internal node] 'Fence' == ? 
        MnWw [Internal node] 'FireplaceQu' == ? 
          Ex [Internal node] 'BsmtFinType1' == ? 
            Unf [Internal node] 'Exterior1st' == ? 
              AsphShn [Leaf node] class=100000
              Wd Sdng [Internal node] 'Exterior2nd' == ? 
                Wd Shng [Internal node] 'Neighborhood' == ? 
                  CollgCr [Internal node] 'HouseStyle' == ? 
                    1Story [Internal node] 'LotFrontage' <= 63.0
                      63.0 [Internal node] 'OverallCond' <= 5.0
                        5.0 [Internal node] 'HeatingQC' == ? 
                          Ex [Internal node] 'RoofStyle' == ? 
                            Hip [Internal node] 'LotConfig' == ? 
                              CulDSac [Internal node] 'BsmtExposure' == ? 
                                No [Internal node] 'BsmtUnfSF' <= 477.5
       

### 